## Model Template
This is an example template where you can insert your model implementation.

**Note:** Customize this template based on your specific model implementation and requirements. Insert your model below.

In [4]:

#import libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import pickle

#load dataset
data = pd.read_csv("creditcard.csv") 

data.head(10) 

#describing the data
print(data.shape) 
print(data.describe()) 

#imbalance in the data
fraud = data[data['Class'] == 1] 
valid = data[data['Class'] == 0] 
outlierFraction = len(fraud)/float(len(valid)) 
print(outlierFraction) 
print('Fraud Cases: {}'.format(len(data[data['Class'] == 1]))) 
print('Valid Transactions: {}'.format(len(data[data['Class'] == 0]))) 

#the amount details for fraudulent transaction
fraud.Amount.describe() 

#the amount details for normal transaction
valid.Amount.describe() 


#separating the X and the Y values
X = data.drop(['Class'], axis = 1) 
Y = data["Class"] 
print(X.shape) 
print(Y.shape) 

xData = X.values 
yData = Y.values 

#training and testing data bifurcation
from sklearn.model_selection import train_test_split 
#split the data into training and testing sets 
xTrain, xTest, yTrain, yTest = train_test_split(xData, yData, test_size = 0.2, random_state = 42) 

#building the Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier 
#random forest model creation 
rfc = RandomForestClassifier() 
rfc.fit(xTrain, yTrain) 
#predictions 
yPred = rfc.predict(xTest) 

from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
acc = accuracy_score(yTest, yPred)
metrics = {
  'accuracy': accuracy_score(yTest, yPred),
  'precision': precision_score(yTest, yPred),
  'recall': recall_score(yTest, yPred),
  'f1score': f1_score(yTest, yPred)
}

feature_importances = rfc.feature_importances_

predictions = rfc.predict(xTest)

model_params = {
    'predictions': predictions,
    'feature_importances': feature_importances,
    'n_estimators': rfc.n_estimators,
    'max_features': rfc.max_features,
    'max_depth': rfc.max_depth if rfc.max_depth is not None else -1
}


#save the model parameters in to a pickle file
with open('model_params.pkl', 'wb') as f:
  pickle.dump(model_params, f)


#save the model metrics in to a text file
with open('metrics.txt', 'w') as f:
  for key, value in metrics.items():
    f.write(f'{key}: {value}\n')

print("Metrics saved:", metrics)


(284807, 31)
                Time            V1            V2            V3            V4  \
count  284807.000000  2.848070e+05  2.848070e+05  2.848070e+05  2.848070e+05   
mean    94813.859575  1.168375e-15  3.416908e-16 -1.379537e-15  2.074095e-15   
std     47488.145955  1.958696e+00  1.651309e+00  1.516255e+00  1.415869e+00   
min         0.000000 -5.640751e+01 -7.271573e+01 -4.832559e+01 -5.683171e+00   
25%     54201.500000 -9.203734e-01 -5.985499e-01 -8.903648e-01 -8.486401e-01   
50%     84692.000000  1.810880e-02  6.548556e-02  1.798463e-01 -1.984653e-02   
75%    139320.500000  1.315642e+00  8.037239e-01  1.027196e+00  7.433413e-01   
max    172792.000000  2.454930e+00  2.205773e+01  9.382558e+00  1.687534e+01   

                 V5            V6            V7            V8            V9  \
count  2.848070e+05  2.848070e+05  2.848070e+05  2.848070e+05  2.848070e+05   
mean   9.604066e-16  1.487313e-15 -5.556467e-16  1.213481e-16 -2.406331e-15   
std    1.380247e+00  1.332271

## Files Required for Backend Processing

The following files need to be generated and saved before sending to the backend:

 `metrics.txt` - Contains model performance metrics: (for both model & data owner)
  - Accuracy
  - Precision
  - Recall
  - F1 Score

 `model_params_encrypted` - Encrypted version of model parameters using Fernet encryption (only for data owner)

 `model_hash` - Contains IPFS hash of the model (only for model owner)

These files needs to be processed and combined into a JSON object containing: (For model Owners)
- model_ipfs_hash
- metrics

These files needs to be processed and combined into a JSON object containing: (For data Owners)
- metrics
- param_ipfs_hash
- param_key (base64 encoded encryption key)

This JSON data is then sent to the backend endpoint at http://127.0.0.1:5000/update-data

Below shown is an example of file generation and backend processing

In [5]:
#Read the saved model metrics
%pip install requests
import json
import requests


with open('metrics.txt', 'r') as file:
  metrics = file.read().strip()

metrics_dict = {}
for line in metrics.splitlines():
  if ': ' in line:
    key, value = line.split(': ', 1)
    metrics_dict[key] = int(float(value) * 100)


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Model Owner Section

**Note:** This section is specifically for model owners. If you are a data owner, please ignore or remove the cell below.

This section handles:
- Model IPFS upload
- Backend data preparation

In [11]:
# #Upload the model on to IPFS
%pip install python-dotenv

import sys
sys.path.append('../..')

from ipfs_configs import upload_file_to_ipfs

model_file_path = 'fraud-detection-model.ipynb'

model_ipfs_hash = upload_file_to_ipfs(model_file_path)

with open('model_hash', 'r') as file:
  ipfs_hash = file.read().strip()

# Create final JSON object
data = {
  'model_ipfs_hash': model_ipfs_hash,
  'metrics': metrics_dict,
}


# Convert to JSON string
json_data = json.dumps(data, indent=2)
print("JSON data:", data)

response = requests.post('http://127.0.0.1:5000/update-data', json=data)


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 2


JSON data: {'model_ipfs_hash': 'QmYsntGpsDWyr2fG8om6fAWGa8uDa16tpNnDZfUC3g4U99', 'metrics': {'accuracy': 99, 'precision': 96, 'recall': 77, 'f1score': 85}}


## Data Owner Section

**Note:** This section is specifically for data owners. (Model Owners please keep section as it is, but comment it out when you run this template)

This section handles:
- Model metrics processing
- Parameter encryption
- IPFS upload
- Backend data preparation

In [ ]:
# #Encrypting Model Parameters & uploading it to IPFS

# %pip install cryptography

# import sys
# sys.path.append('..')

# from ipfs_configs import upload_file_to_ipfs
# from cryptography.fernet import Fernet

# key = Fernet.generate_key()
# cipher_suite = Fernet(key)

# with open('model_params.pkl', 'rb') as file:
#   original_data = file.read()

# encrypted_data = cipher_suite.encrypt(original_data)

# with open('model_params_encrypted.pkl', 'wb') as file:
#   file.write(encrypted_data)

# params_file_path = 'model_params_encrypted.pkl'

# param_ipfs_hash = upload_file_to_ipfs(params_file_path)

# import base64

# #Create final JSON object
# data = {
#   'metrics': metrics_dict,
#   'param_ipfs_hash': param_ipfs_hash,
#   'param_key': base64.b64encode(key).decode('utf-8')
# }


# # Convert to JSON string
# json_data = json.dumps(data, indent=2)
# print("JSON data:", data)

# response = requests.post('http://127.0.0.1:5000/update-data', json=data)


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
JSON data: {'metrics': {'accuracy': 99, 'precision': 91, 'recall': 75, 'f1score': 83}, 'param_ipfs_hash': 'QmYJkXGkL3sazJrppiotW7txpL3ocvHqS9HzPdNTfvL68R', 'param_key': 'cWRLSFhuckU1UnVOVHRSWUVtaDhqZzJfSnFpTzBldGNFUy13ZjM2OU82RT0='}
